In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train_csv_path = r'C:\Users\TDInstaller\Documents\Kaggle\titanic\train.csv'
test_csv_path = r'C:\Users\TDInstaller\Documents\Kaggle\titanic\test.csv'
raw_data_train = pd.read_csv(train_csv_path)
raw_data_test = pd.read_csv(test_csv_path)

titanic_data_train = raw_data_train.copy()
titanic_data_test = raw_data_test.copy()
titanic_data_train ['Source_File'] = 'Train'
titanic_data_test ['Source_File'] = 'Test'
titanic_data = pd.concat([titanic_data_train,titanic_data_test])

In [4]:
titanic_data_copy = titanic_data.copy()

# Exploring Source Variables

In [ ]:
titanic_data.info()

In [ ]:
titanic_data.head()

In [ ]:
titanic_data.describe() #for numerical columns

* Seems like ~38.4% of passengers survived
* Most of the passengers were in 3rd class
* Average age is ~30 years
* SibSp/Parch seems to be mostly zeros i.e. most passengers were travelling alone
* Fare values are higly varied (high variance)

# Data Variables Vs Survival

In [ ]:
sns.pairplot(titanic_data)

## Sex Vs Survival

In [ ]:
sns.barplot(data=titanic_data, x='Sex', y='Survived')

In [ ]:
sns.countplot(data=titanic_data, x='Sex', hue='Survived')

In [ ]:
titanic_data['Sex'].value_counts()

In [ ]:
pd.pivot_table(data=titanic_data, columns='Sex', index='Survived', values= 'PassengerId' ,aggfunc='count', margins=True)
#male passengers were more than female ones, but more females survived than males (women and children first) 

In [ ]:
titanic_data[['Survived','Sex']].groupby('Sex').mean().sort_values(by='Survived', ascending=False)['Survived']*100
#order by survived so 1= Survived will represent the first row
#~74% of females survived
#~19% of males survived

In [ ]:
t = titanic_data[['PassengerId','Sex']].groupby('Sex').count().reset_index()
t['%'] = t['PassengerId']/len(titanic_data)*100
t
#~64% of passengers were males

## Embarked Vs Survival 

In [ ]:
sns.barplot(data=titanic_data, x='Embarked', y='Survived')

In [ ]:
sns.countplot(data=titanic_data, x='Embarked', hue='Survived')

In [ ]:
pd.pivot_table(data=titanic_data, columns='Embarked', index='Survived', values= 'PassengerId' ,aggfunc='count', margins=True )
#most pple who survived had departed from South Hampton, but most passengers actually departed from there anyway, 
#and actually, majority of pple who departed from South Hampton didn't survive
#people who departed from Cherbourg are more likely to survive than not

In [ ]:
# seems like C are more likely to survive indeed
titanic_data[["Embarked", "Survived"]].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

## Cabin, Pclass Vs Survival 

In [ ]:
titanic_data['Cabin'].unique()

In [ ]:
titanic_data['cabin_cat_series'] = titanic_data['Cabin'].apply(lambda x:str(x)[0])
pd.pivot_table(data=titanic_data, columns='cabin_cat_series', index='Survived', values= 'PassengerId' ,aggfunc='count', margins=True )
#most pple have unknown cabins 687/891 passengers
#most unknown cabiners didn't survive

In [ ]:
titanic_data.groupby(['cabin_cat_series','Pclass'])['PassengerId'].count()
#majority of unknown cabins are in class 3 

In [ ]:
pd.pivot_table(data=titanic_data, columns='Pclass', index='Survived', values= 'PassengerId' ,aggfunc='count', margins=True )
# majority of class 3 sdidn't survive 
# conc seems like most unknown cabiners didn't survice, most of the unknowns were class 3, and majority of class 3 didn't survive

In [ ]:
titanic_data['cabin_unknown']=titanic_data['cabin_cat_series'].apply(lambda x: 1 if x =='n' else 0)
sns.countplot(data=titanic_data ,x='cabin_unknown') #most passengers have unknown cabins

In [ ]:
sns.countplot(data=titanic_data ,x='cabin_unknown', hue='Survived') #unknown cabiners are less likely to survive

In [ ]:
sns.countplot(data=titanic_data ,x='Pclass') #most passengers are 3rd class

In [ ]:
sns.countplot(data=titanic_data ,x='Pclass', hue='Survived') #first class are more likely to survive

In [ ]:
sns.countplot(data=titanic_data ,x='cabin_unknown', hue='Pclass') #unkown cabiners are more likely to be 3rd class which is least likely to survive

## SibSp Vs Survival

In [ ]:
pd.pivot_table(data=titanic_data, columns='SibSp', index='Survived', values= 'PassengerId' ,aggfunc='count', margins=True )
#most passengers had 0 or 1 SibSP (already deduced before that most passengers travelled alone)
#people with 0 or 1 siblings and spouses had higher chance of survival compared to pple with more sib/sps. 
#However, we can also see that only 210/608 pple who traveled alone survived
#looks like majority of passengers traveled alone and majority of passegers didn't survive,which are two things we already know
#but overall if a passenger is travelling alone, they would have higher chance to survive than otherwise

In [ ]:
sns.countplot(data=titanic_data, x='SibSp')

In [ ]:
sns.countplot(data=titanic_data, x='SibSp', hue='Survived')

In [ ]:
sns.barplot(data=titanic_data, x='SibSp', y='Survived')

## Parch Vs Survival

In [ ]:
pd.pivot_table(data=titanic_data, columns='Parch', index='Survived', values= 'PassengerId' ,aggfunc='count', margins=True )
#looks like travelling with 0 or 1 parets/children gives you a higher chance of survival than otherwise

In [ ]:
sns.countplot(data=titanic_data, x='Parch', hue='Survived')

In [ ]:
sns.barplot(data=titanic_data, x='Parch', y='Survived')

## Ticket Vs Survival

In [ ]:
list(titanic_data.Ticket.unique()) #some tickets are numeric and some are alphanumeric

In [ ]:
titanic_data.Ticket.nunique()/titanic_data.Ticket.shape[0]*100
#~71% of tickets are unique, so  may be some tickets have multiple passangers on them

In [ ]:
titanic_data['numeric_Ticket'] = titanic_data['Ticket'].apply(lambda x:1 if x.isnumeric() else 0)

In [ ]:
pd.pivot_table(data=titanic_data, columns='numeric_Ticket', index='Survived', values= 'PassengerId' ,aggfunc='count', margins=True )
#majority of tickets were numeric, majority representation makes it so that majority of survivals had numeric tickets 
#and majority of those whose didn't survive are also holders of numeric tickets

In [ ]:
sns.countplot(data=titanic_data, x='numeric_Ticket')

In [ ]:
sns.countplot(data=titanic_data, x='numeric_Ticket', hue='Survived') # doesn't seem like eother type would inc likelihood of survival

In [ ]:
sns.barplot(data=titanic_data, x='numeric_Ticket', y='Survived')

In [ ]:
sns.countplot(data=titanic_data, x='numeric_Ticket', hue='Pclass')

## Age, class Vs survival

In [ ]:
f, axes = plt.subplots(1,1, figsize = (16, 5))
sns.distplot(titanic_data.Age);

In [ ]:
sns.boxplot(data=titanic_data, x='Pclass', y='Age') #3rd class are the youngest

In [ ]:
g = sns.FacetGrid(titanic_data, col='Pclass')
g.map(plt.hist, 'Age', bins=60)

In [ ]:
g = sns.FacetGrid(titanic_data, col='Survived')
g.map(plt.hist, 'Age', bins=60)

In [3]:
#sort the ages into age groups #explore more about binning!
bins = [0, 2, 12, 17, 60, np.inf]
labels = ['baby', 'child', 'teenager', 'adult', 'elderly']
age_groups = pd.cut(titanic_data.Age, bins, labels = labels)
titanic_data['Age_Group'] = age_groups

In [ ]:
sns.countplot(data=titanic_data, x='Age_Group',hue='Survived') #babies and children are more likely to survive

In [ ]:
sns.countplot(data=titanic_data, x='Age_Group',hue='Pclass')

In [ ]:
sns.barplot(data=titanic_data,x="Age_Group", y="Survived")

## Fare vs Survival

In [ ]:
f, axes = plt.subplots(1,1, figsize = (16, 5))
sns.distplot(titanic_data.Fare);

In [ ]:
sns.distplot(titanic_data.Fare);

In [ ]:
survived_fares = titanic_data[titanic_data['Survived']==1][['Fare','Survived']]
not_survived_fares = titanic_data[titanic_data['Survived']==0][['Fare','Survived']]

In [ ]:
plt.figure(figsize = (16, 4))
sns.distplot(survived_fares.Fare);
sns.distplot(not_survived_fares.Fare);

In [ ]:
plt.figure(figsize = (16, 4))
sns.distplot(survived_fares.Fare, label='Survived=1');
sns.distplot(not_survived_fares.Fare, label='Survived=0');
plt.legend()

In [ ]:
f, axes = plt.subplots(1,1, figsize = (14, 3))
sns.distplot(survived_fares.Fare).set_title('Fares Dist For Survivors');

In [ ]:
f, axes = plt.subplots(1,1, figsize = (14, 3))
sns.distplot(not_survived_fares.Fare).set_title('Fares Dist For Non-Survivors');

In [ ]:
# Higher fare correlates with higher survival rate
g = sns.FacetGrid(titanic_data, col='Survived')
g.map(plt.hist, 'Fare', bins=30)

# Feature Engineering

In [8]:
titanic_data = titanic_data_copy.copy()

## Age

In [9]:
def get_class_avgAge():
    return titanic_data.groupby('Pclass')['Age'].mean()

def derive_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        return get_class_avgAge()[Pclass]
    else:
        return Age

titanic_data['Age'] = titanic_data[['Age','Pclass']].apply(derive_age,axis=1)  

bins = [0, 2, 12, 17, 60, np.inf]
labels = ['baby', 'child', 'teenager', 'adult', 'elderly']
age_groups = pd.cut(titanic_data.Age, bins, labels = labels)
titanic_data['Age_Group'] = age_groups


## Title, Cabin

In [10]:

titanic_data['passenger_title'] = titanic_data.Name.apply(lambda x: x.split(',',1)[1].split('.')[0].strip()).values

titanic_data['cabin_cat'] = titanic_data.Cabin.apply(lambda x: str(x)[0])
titanic_data['cabin_multiple'] = titanic_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))

## Ticket

In [11]:
titanic_data['numeric_ticket'] = titanic_data.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
titanic_data['Ticket_Frequency'] = titanic_data.groupby('Ticket')['PassengerId'].count()

def get_all_tickets_frequency():
        return titanic_data.groupby('Ticket')['PassengerId'].count()
def get_single_ticket_frequency(cols):
    Ticket = cols[0]
    return get_all_tickets_frequency()[str(Ticket)]   
titanic_data['Ticket_Frequency'] = titanic_data[['Ticket']].apply(get_single_ticket_frequency,axis=1)

## Fare

In [12]:
def get_class_avgFare():
    return titanic_data.groupby('Pclass')['Fare'].mean()
    
def derive_Fare(cols):
    Fare = cols[0]
    Pclass = cols[1]
    if pd.isnull(Fare):
        return get_class_avgFare()[Pclass]
    else:
        return Fare   
    
titanic_data['Fare'] = titanic_data[['Fare','Pclass']].apply(derive_Fare,axis=1)    

## Final na handeling and data types handling

In [13]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   PassengerId       1309 non-null   int64   
 1   Survived          891 non-null    float64 
 2   Pclass            1309 non-null   int64   
 3   Name              1309 non-null   object  
 4   Sex               1309 non-null   object  
 5   Age               1309 non-null   float64 
 6   SibSp             1309 non-null   int64   
 7   Parch             1309 non-null   int64   
 8   Ticket            1309 non-null   object  
 9   Fare              1309 non-null   float64 
 10  Cabin             295 non-null    object  
 11  Embarked          1307 non-null   object  
 12  Source_File       1309 non-null   object  
 13  Age_Group         1309 non-null   category
 14  passenger_title   1309 non-null   object  
 15  cabin_cat         1309 non-null   object  
 16  cabin_multiple    1309 no

In [23]:
titanic_data.dropna(subset=['Embarked'],inplace = True)
titanic_data.Pclass = titanic_data.Pclass.astype(str)
titanic_data.numeric_ticket = titanic_data.numeric_ticket.astype(str)
titanic_data['Age'] = round(titanic_data['Age'], 2)

In [24]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1307 entries, 0 to 417
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   PassengerId       1307 non-null   int64   
 1   Survived          889 non-null    float64 
 2   Pclass            1307 non-null   object  
 3   Name              1307 non-null   object  
 4   Sex               1307 non-null   object  
 5   Age               1307 non-null   float64 
 6   SibSp             1307 non-null   int64   
 7   Parch             1307 non-null   int64   
 8   Ticket            1307 non-null   object  
 9   Fare              1307 non-null   float64 
 10  Cabin             293 non-null    object  
 11  Embarked          1307 non-null   object  
 12  Source_File       1307 non-null   object  
 13  Age_Group         1307 non-null   category
 14  passenger_title   1307 non-null   object  
 15  cabin_cat         1307 non-null   object  
 16  cabin_multiple    1307 no

In [28]:
#Added age groups instead of age
all_dummies = pd.get_dummies(titanic_data[['Pclass','Sex','Age_Group','SibSp','Fare','Parch','Embarked','Ticket_Frequency','cabin_cat','cabin_multiple','numeric_ticket','passenger_title','Source_File']])

In [29]:
all_dummies.columns

Index(['SibSp', 'Fare', 'Parch', 'Ticket_Frequency', 'cabin_multiple',
       'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_Group_baby', 'Age_Group_child', 'Age_Group_teenager',
       'Age_Group_adult', 'Age_Group_elderly', 'Embarked_C', 'Embarked_Q',
       'Embarked_S', 'cabin_cat_A', 'cabin_cat_B', 'cabin_cat_C',
       'cabin_cat_D', 'cabin_cat_E', 'cabin_cat_F', 'cabin_cat_G',
       'cabin_cat_T', 'cabin_cat_n', 'numeric_ticket_0', 'numeric_ticket_1',
       'passenger_title_Capt', 'passenger_title_Col', 'passenger_title_Don',
       'passenger_title_Dona', 'passenger_title_Dr',
       'passenger_title_Jonkheer', 'passenger_title_Lady',
       'passenger_title_Major', 'passenger_title_Master',
       'passenger_title_Miss', 'passenger_title_Mlle', 'passenger_title_Mme',
       'passenger_title_Mr', 'passenger_title_Mrs', 'passenger_title_Ms',
       'passenger_title_Rev', 'passenger_title_Sir',
       'passenger_title_the Countess', 'Source_File_Test'

## **Splitting train_csv from Test_csv**

In [30]:
X_train_data = all_dummies[all_dummies.Source_File_Train == 1].drop(['Source_File_Train','Source_File_Test'], axis =1)
y_train_data = titanic_data['Survived'].dropna()

train_csv = pd.concat([X_train_data,y_train_data], axis=1)

X_test = all_dummies[all_dummies.Source_File_Train == 0].drop(['Source_File_Train','Source_File_Test'], axis =1)
test_csv = X_test.copy()

In [31]:
train_csv.head()

,SibSp,Fare,Parch,Ticket_Frequency,cabin_multiple,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,passenger_title_Miss,passenger_title_Mlle,passenger_title_Mme,passenger_title_Mr,passenger_title_Mrs,passenger_title_Ms,passenger_title_Rev,passenger_title_Sir,passenger_title_the Countess,Survived
0,1,7.2500,0,1,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0.0
1,1,71.2833,0,2,1,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1.0
2,0,7.9250,0,1,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1.0
3,1,53.1000,0,2,1,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1.0
4,0,8.0500,0,1,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0.0


In [32]:
train_csv.describe()

,SibSp,Fare,Parch,Ticket_Frequency,cabin_multiple,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,passenger_title_Miss,passenger_title_Mlle,passenger_title_Mme,passenger_title_Mr,passenger_title_Mrs,passenger_title_Ms,passenger_title_Rev,passenger_title_Sir,passenger_title_the Countess,Survived
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,...,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,0.524184,32.096681,0.382452,2.121485,0.265467,0.240720,0.206974,0.552306,0.350956,0.649044,...,0.203600,0.002250,0.001125,0.581552,0.139483,0.001125,0.006749,0.001125,0.001125,0.382452
std,1.103705,49.697504,0.806761,1.798673,0.546642,0.427761,0.405365,0.497536,0.477538,0.477538,...,0.402901,0.047404,0.033539,0.493582,0.346644,0.033539,0.081922,0.033539,0.033539,0.486260
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,7.895800,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,14.454200,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,31.000000,0.000000,3.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,8.000000,512.329200,6.000000,11.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Scaling

In [33]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
train_inputs = X_train_data.copy()
#train_inputs[['Age','SibSp','Parch','Fare','cabin_multiple','Ticket_Frequency']]= scale.fit_transform(train_inputs[['Age','SibSp','Parch','Fare','cabin_multiple','Ticket_Frequency']])
train_inputs[['SibSp','Parch','Fare','cabin_multiple','Ticket_Frequency']]= scale.fit_transform(train_inputs[['SibSp','Parch','Fare','cabin_multiple','Ticket_Frequency']])
scaled_training_inputs = train_inputs.copy()

In [34]:
targets = train_csv['Survived']

In [36]:
test_csv[['SibSp','Parch','Fare','cabin_multiple','Ticket_Frequency']]= scale.fit_transform(test_csv[['SibSp','Parch','Fare','cabin_multiple','Ticket_Frequency']])
#test_csv[['Age','SibSp','Parch','Fare','cabin_multiple','Ticket_Frequency']]= scale.fit_transform(test_csv[['Age','SibSp','Parch','Fare','cabin_multiple','Ticket_Frequency']])
X_test_scaled = test_csv.copy()

## Split train csv data into train and test

In [37]:
from sklearn.model_selection import train_test_split
# declare 4 variables for the split
x_train, x_test, y_train, y_test = train_test_split(scaled_training_inputs, targets, #train_size = 0.8, 
                                                                            test_size = 0.2, random_state = 20)

## ML

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [39]:
from sklearn.model_selection import cross_val_score
lr2 = LogisticRegression()
cv = cross_val_score(lr2,x_train,y_train,cv=10)
print(cv)
print(cv.mean())

[0.81944444 0.81690141 0.8028169  0.81690141 0.88732394 0.83098592
 0.78873239 0.81690141 0.78873239 0.81690141]
0.8185641627543037


In [40]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 1,min_samples_split=5)
cv = cross_val_score(rf,x_train,y_train,cv=10)
print(cv)
print(cv.mean())

[0.76388889 0.8028169  0.76056338 0.78873239 0.87323944 0.81690141
 0.8028169  0.77464789 0.78873239 0.85915493]
0.8031494522691706


In [41]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state =1,learning_rate =0.2,use_label_encoder=False)
cv = cross_val_score(xgb,x_train,y_train,cv=10, verbose=0)
#print(cv)
print(cv.mean())

[13:14:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:14:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:14:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:14:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [42]:
eval_set = [(x_test, y_test)]
xgb.fit(x_train, y_train, eval_set=eval_set, early_stopping_rounds=10)
xgb.score(x_test, y_test)

[13:14:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.60064
[1]	validation_0-logloss:0.54381
[2]	validation_0-logloss:0.50453
[3]	validation_0-logloss:0.47662
[4]	validation_0-logloss:0.45744
[5]	validation_0-logloss:0.44434
[6]	validation_0-logloss:0.43762
[7]	validation_0-logloss:0.42924
[8]	validation_0-logloss:0.42690
[9]	validation_0-logloss:0.42547
[10]	validation_0-logloss:0.42480
[11]	validation_0-logloss:0.42559
[12]	validation_0-logloss:0.42530
[13]	validation_0-logloss:0.42567
[14]	validation_0-logloss:0.42370
[15]	validation_0-logloss:0.42495
[16]	validation_0-logloss:0.42524
[17]	validation_0-logloss:0.42532
[18]	validation_0-logloss:0.42453
[19]	validation_0-logloss:0.42290
[20]	validation_0-lo

0.8314606741573034

In [43]:
xgb_predictions = xgb.predict(X_test_scaled)

In [44]:
submission_xgb = pd.DataFrame({
        "PassengerId": raw_data_test['PassengerId'],
       "Survived": xgb_predictions
  })
submission_xgb.to_csv('xgb_predictions_sumission7Feb.csv',  index = False)